In [ ]:
# !pip install fastf1

In [ ]:
!python -V 

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import matplotlib
import warnings
from datetime import datetime, date, time
import ipywidgets as widgets
import pdb
# import fastf1
import glob
import os
import getpass
import sys


print('Libraries imported')

In [ ]:
# Configure Jupyter Notebook
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500) 
pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', -1)
display(HTML("<style>div.output_scroll { height: 35em; }</style>"))

reload(plt)
%matplotlib inline
%config InlineBackend.figure_format ='retina'

warnings.filterwarnings('ignore')

print('Notebook configured')

In [ ]:
df_drivers = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv')

In [ ]:
df_laptimes = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/lap_times.csv')

In [ ]:
df_races = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/races.csv')

In [ ]:
df_pitstops = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv')

In [ ]:
df_results = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/results.csv')

In [ ]:
df = pd.merge(df_laptimes,df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df = pd.merge(df,df_races[['raceId','name','date','year']],how='left',on='raceId')

In [ ]:
df['time']=pd.to_timedelta(df['milliseconds'], unit='ms')
df['seconds']=df['milliseconds']/1000

In [ ]:
df.head()

In [ ]:
from tqdm import tqdm

In [ ]:
df_laps = df[(df['year']==2021)&((df['code']=='VER')|(df['code']=='HAM'))].copy()
df_laps.rename(columns={'position':'lap position'},inplace=True)
df_laps = df_laps.merge(df_results[['raceId','driverId','position']],how='left',on=['raceId','driverId'])
df_laps = df_laps.merge(df_pitstops[['raceId','driverId','lap','stop']],how='left',on=['raceId','driverId','lap'])
df_laps['stop'].fillna(0,inplace=True)
df_laps['stop']=df_laps['stop'].astype(int)
df_laps['stop'][df_laps['stop']==0] = ''

In [ ]:
df_laps.head()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=10,5 
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12

plt.figure(figsize=(10, 100))
for i, f in tqdm(enumerate(df_laps['name'].unique())):
    try:
        HAM_pos = list(set(df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')]['position']))[0]
        if HAM_pos == r'\N':
            HAM_pos = 'DNF'
    except:
        HAM_pos = 'DNS'
    try:    
        VER_pos = list(set(df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')]['position']))[0]
        if VER_pos == r'\N':
            VER_pos = 'DNF'
    except:
        VER_pos = 'DNS'
    fig,ax = plt.subplots(1,1)
    plt.title(f)
    sns.lineplot(data=df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')],
                 x='lap', 
                 y='seconds',
                 hue='code',
                 palette=['black'],
                 ax=ax,
                 marker='.',
                 # marker_size=3
                )
    HAM_stops = df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')]
    for j,label in enumerate(HAM_stops['stop']):
        plt.annotate(label, (HAM_stops['lap'].iloc[j], 
                             HAM_stops['seconds'].iloc[j]
                            ), 
                     color = 'black',
                     bbox=dict(boxstyle="circle,pad=0", fc="white", ec="black", lw=0.5),
                     ha="center", va="center",
                    )
    
    sns.lineplot(data=df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')],
                 x='lap', 
                 y='seconds',
                 hue='code',
                 ax=ax,
                 palette = ['orange'],
                 marker='.',
                )
    
    VER_stops = df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')]
    for j,label in enumerate(VER_stops['stop']):
        plt.annotate(label, (VER_stops['lap'].iloc[j], 
                             VER_stops['seconds'].iloc[j]
                            ),
                     color='orange',
                     bbox=dict(boxstyle="circle,pad=0", fc="white", ec="orange", lw=0.5),
                     ha="center", va="center",
                    )
    
    if ax.get_legend_handles_labels()[1][0]=='HAM':
        plt.legend(['HAM: P-'+str(HAM_pos), 'VER: P-'+str(VER_pos)])
        leg = ax.get_legend()
        leg.legendHandles[0].set_color('black')
        leg.legendHandles[1].set_color('orange')
    elif ax.get_legend_handles_labels()[1][0]=='VER':
        plt.legend(['VER: P-'+str(VER_pos), 'HAM: P-'+str(HAM_pos)])
        leg = ax.get_legend()
        leg.legendHandles[0].set_color('orange')
        leg.legendHandles[1].set_color('black')
        
        
plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=10,5 
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12

plt.figure(figsize=(10, 100))
for i, f in tqdm(enumerate(df_laps['name'].unique())):
    try:
        HAM_pos = list(set(df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')]['position']))[0]
        if HAM_pos == r'\N':
            HAM_pos = 'DNF'
    except:
        HAM_pos = 'DNS'
    try:    
        VER_pos = list(set(df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')]['position']))[0]
        if VER_pos == r'\N':
            VER_pos = 'DNF'
    except:
        VER_pos = 'DNS'
    fig,ax = plt.subplots(1,1)
    plt.title(f)
    sns.lineplot(data=df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')],
                 x='lap', 
                 y='lap position',
                 hue='code',
                 palette=['black'],
                 marker='.',
                 ax=ax
                )
    HAM_stops = df_laps[(df_laps['name']==f)&(df_laps['code']=='HAM')]
    for j,label in enumerate(HAM_stops['stop']):
        plt.annotate(label, (HAM_stops['lap'].iloc[j], 
                             HAM_stops['lap position'].iloc[j]
                            ), 
                     color = 'black',
                     bbox=dict(boxstyle="circle,pad=0", fc="white", ec="black", lw=0.5),
                     ha="center", va="center",
                    )
    
    sns.lineplot(data=df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')],
                 x='lap', 
                 y='lap position',
                 hue='code',
                 ax=ax,
                 palette = ['orange'],
                 marker='.',
                )
    
    VER_stops = df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')]
    for j,label in enumerate(VER_stops['stop']):
        plt.annotate(label, (VER_stops['lap'].iloc[j], 
                             VER_stops['lap position'].iloc[j]
                            ),
                     color='orange',
                     bbox=dict(boxstyle="circle,pad=0", fc="white", ec="orange", lw=0.5),
                     ha="center", va="center",
                    )
    
    if ax.get_legend_handles_labels()[1][0]=='HAM':
        plt.legend(['HAM: P-'+str(HAM_pos), 'VER: P-'+str(VER_pos)])
        leg = ax.get_legend()
        leg.legendHandles[0].set_color('black')
        leg.legendHandles[1].set_color('orange')
    elif ax.get_legend_handles_labels()[1][0]=='VER':
        plt.legend(['VER: P-'+str(VER_pos), 'HAM: P-'+str(HAM_pos)])
        leg = ax.get_legend()
        leg.legendHandles[0].set_color('orange')
        leg.legendHandles[1].set_color('black')
    plt.ylim(reversed(plt.ylim()))    
    # 
    # VER_stop = df_laps[(df_laps['name']==f)&(df_laps['code']=='VER')]
    # for j,label in enumerate(df_laps['stop']):
    #     plt.annotate(label, (df_laps['lap'][j], 
    #                          df_laps['seconds'][j]
    #                         ) 
    #                 )
        
plt.tight_layout()
plt.show()

In [ ]:
df_results = df_results.merge(df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df_results = df_results.merge(df_races[['raceId','name','date','year']],how='left',on='raceId')

In [ ]:
df_results.drop_duplicates(inplace=True)

In [ ]:
df_results[df_results['position']==r'\N']=0
df_results['position'] = df_results['position'].astype(int)

In [ ]:
df_results = df_results[(df_results['year']==2021)&((df_results['code']=='VER')|(df_results['code']=='HAM'))]

In [ ]:
df_results.head()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,10 
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.barplot(data=df_results,x='position', y='name',hue='code',palette=['black','orange'])
plt.tight_layout()
plt.show()

In [ ]:
df_driver_standings = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/driver_standings.csv')
df_driver_standings.head()

In [ ]:
df_driver_standings = df_driver_standings.merge(df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df_driver_standings = df_driver_standings.merge(df_races[['raceId','name','date','year']],how='left',on='raceId')

In [ ]:
df_driver_standings.head()

In [ ]:
df_driver_standings = df_driver_standings[(df_driver_standings['year']==2021)&((df_driver_standings['code']=='VER')|(df_driver_standings['code']=='HAM'))]
df_driver_standings.sort_values(['date'],inplace=True)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=10,6 
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.lineplot(data=df_driver_standings,x='name', y='points',hue='code',marker='.',palette=['black','orange'])
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_pitstops = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv')
df_pitstops = df_pitstops.merge(df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df_pitstops = df_pitstops.merge(df_races[['raceId','name','date','year']],how='left',on='raceId')
df_pitstops = df_pitstops[(df_pitstops['year']==2021)&((df_pitstops['code']=='VER')|(df_pitstops['code']=='HAM'))]
df_pitstops['duration']=df_pitstops['milliseconds']/1000
# df_pitstops.head()

In [ ]:
# average duration of a pitstop - I've taken out the long pitstops as these are anomalies
df_pitstops[df_pitstops['duration']<40].groupby(['code']).mean()['duration']

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,12
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.barplot(data=df_pitstops[df_pitstops['duration']<40].groupby(['name','code']).mean().reset_index(),
            y='name', 
            x='duration',
            hue='code',
            palette=['black','orange'])
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_pitstops = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/pit_stops.csv')
df_pitstops = df_pitstops.merge(df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df_pitstops = df_pitstops.merge(df_races[['raceId','name','date','year']],how='left',on='raceId')
df_pitstops = df_pitstops[(df_pitstops['year']>2010)&((df_pitstops['code']=='VER')|(df_pitstops['code']=='HAM'))]
df_pitstops['duration']=df_pitstops['milliseconds']/1000
# df_pitstops.dropna(inplace=True)
# df_pitstops.head()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,18 
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.violinplot(data=df_pitstops[df_pitstops['duration']<200],
               y='name', 
               x='duration',
               hue='code',
               split=True,
               inner="stick",
               palette=['black','orange'],
              )
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_qualifying = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/qualifying.csv')
df_qualifying = df_qualifying.merge(df_drivers[['driverId','code','driverRef']],how='left',on='driverId')
df_qualifying = df_qualifying.merge(df_races[['raceId','name','date','year']],how='left',on='raceId')
df_qualifying = df_qualifying[(df_qualifying['year']==2021)&((df_qualifying['code']=='VER')|(df_qualifying['code']=='HAM'))]

In [ ]:
df_conversions = df_qualifying.copy()
df_conversions.rename(columns={'position':'start position'},inplace=True)
df_first_lap = df_laps[df_laps['lap']==1]
df_conversions = df_conversions.merge(df_first_lap[['raceId','driverId','lap position']],on=['raceId','driverId'],how='left')
df_conversions.rename(columns={'lap position':'lap 1 position'}, inplace=True)
df_conversions = df_conversions.merge(df_results[['raceId','driverId','position']],on=['raceId','driverId'],how='left')
df_conversions.rename(columns={'position':'final position'},inplace=True)
df_conversions['Start to Lap 1']= -df_conversions['lap 1 position'] + df_conversions['start position']
df_conversions['Qualifying conversion']= -df_conversions['final position'] + df_conversions['start position']
df_conversions['Lap 1 conversion']= -df_conversions['final position'] + df_conversions['lap 1 position']

In [ ]:
df_conversions.head()

In [ ]:
df_conversions.groupby(['code']).mean()[['Start to Lap 1','Qualifying conversion','Lap 1 conversion']]

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,12
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.barplot(data=df_conversions,
            y='name', 
            x='Start to Lap 1',
            hue='code',
            palette=['orange','black'])
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,12
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.barplot(data=df_conversions,
            y='name', 
            x='Qualifying conversion',
            hue='code',
            palette=['orange','black'])
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize']=8,12
plt.rcParams['font.family'] = 'Calibri'
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.labelsize']=12
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.rcParams['legend.fontsize']=12
sns.barplot(data=df_conversions,
            y='name', 
            x='Lap 1 conversion',
            hue='code',
            palette=['orange','black'])
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

The championship, though shrouded in controversy over its victor, provides some interesting information that we can conclude to be:

- Hamilton typically outperforms Verstappen in qualifying sessions.
- Verstappen tends to excel during races, steadily advancing through the field. Meanwhile, Hamilton tends to maintain his initial position.
- In 2021, Hamilton's pit stops were consistently about half a second faster than Verstappen's, accounting for outliers. While penalties might have played a role, it's evident that Red Bull's performance in this regard fell short of Mercedes.
- Hamilton's lap times displayed greater consistency at the start of the season, while Verstappen's performance seemed more erratic. However, as the season progressed, Verstappen's consistency improved, whereas Hamilton's lap times became more variable in several races.
- Notably, Mercedes boasts a higher top speed compared to Red Bull. Consequently, Verstappen had to aggressively utilize the curbs to close the gap on Hamilton.